# 0. Initialisation

In [119]:
import time
import random
import numpy as np
import robotic as ry
import open3d as o3d
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor

# some important constants
i_just_want_to_see_the_BotOp_simulation = True

want_to_see_uncropped_point_cloud = False
want_to_see_cropped_point_cloud = not want_to_see_uncropped_point_cloud
want_to_see_normals_of_objects = True
want_to_see_separated_clusters = True
want_to_see_best_grasp_for_each_object = True
  
  
numOfObjects = 4
pclColor = [0,0,255]
botOpSpeed = 1
voxelSize = 0.006
degToleration = 6

if i_just_want_to_see_the_BotOp_simulation:
  want_to_see_uncropped_point_cloud, want_to_see_cropped_point_cloud, want_to_see_normals_of_objects, want_to_see_separated_clusters, want_to_see_best_grasp_for_each_object = False, False, False, False, False


C = ry.Config()
C.addFile('/home/vboxuser/Desktop/CS449/Migi/hw3/GFiles-HW3/environment.g')

qHome = C.getJointState()
C.view()

0

In [120]:
# Additional object: a ball

ballPos = C.getFrame("bin").getPosition()
ballPos[1] = ballPos[1] + 0.2
ballPos[2] = ballPos[2] + 0.075

ball = C.addFrame("ball")
ball.setShape(ry.ST.sphere, [.03])
ball.setPosition(ballPos)
ball.setMass(0.1)
ball.setContact(1)

C.view()

0

In [121]:
bot = ry.BotOp(C, useRealRobot=False)

In [122]:
try:
  del bot
  del komo
except:
  pass

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


# 1.1 Perception Module

### Get point clouds for each camera

In [123]:
numOfCameras = 4
cam = [None] * numOfCameras
rgb = [None] * numOfCameras
depth = [None] * numOfCameras
pcl = [None] * numOfCameras


for i in range(numOfCameras):
  cam[i] = ry.CameraView(C)
  cam[i].setCamera('camera'+str(i+1))
  rgb[i], depth[i] = cam[i].computeImageAndDepth(C)
  pcl[i] = ry.depthImage2PointCloud(depth[i], cam[i].getFxycxy())
  print(pcl[i].shape)

(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)


### Now normalise the points relative to world frame

In [124]:
for i in range(numOfCameras):
  curCam = C.getFrame("camera"+str(i+1))  
  curCamRotMat = curCam.getRotationMatrix()
  curCamPos = curCam.getPosition()
    
  for h in range(pcl[i].shape[0]):    # Iterate over height
    for w in range(pcl[i].shape[1]):  # Iterate over width
      point = pcl[i][h, w]            # Get the 3D point [x, y, z]
     
      # Apply rotation
      rotated_point = np.dot(curCamRotMat, point)  # Rotate the point
      
      # Apply translation
      transformed_point = rotated_point + curCamPos  # Add the camera position
      
      pcl[i][h, w] = transformed_point      
      

# Visualize the point cloud
if want_to_see_uncropped_point_cloud:
  pclFrames = [None] * numOfCameras

  for i in range(numOfCameras):
    pclFrames[i] = C.addFrame('pcl'+str(i+1), 'world')
    pclFrames[i].setPointCloud(pcl[i], pclColor)

  C.view()
  pclFrames = None

### Now we extract the ones inside the bounding box

In [125]:
centre = C.getFrame("bin").getPosition()
centre[2] = centre[2] + 0.087  # Raise slightly to remove bin surface

off = .1
offsets = [0.3, 0.25, 0.05]

bounding_box_min_x = centre[0] - offsets[0]
bounding_box_max_x = centre[0] + offsets[0]

bounding_box_min_y = centre[1] - offsets[1]
bounding_box_max_y = centre[1] + offsets[1]

bounding_box_min_z = centre[2] - offsets[2]
bounding_box_max_z = centre[2] + offsets[2]


for i in range(numOfCameras):
  for h in range(pcl[i].shape[0]):  
    for w in range(pcl[i].shape[1]):  
      point = pcl[i][h, w] 
      eachX, eachY, eachZ = point  
      
      # Check if the point is within the bounding box
      if not (bounding_box_min_x < eachX < bounding_box_max_x and
              bounding_box_min_y < eachY < bounding_box_max_y and
              bounding_box_min_z < eachZ < bounding_box_max_z):
          pcl[i][h, w] = [0, 0, 0]


### Now combine pcl from the 4 cameras and view the final bounded point cloud

In [126]:

if want_to_see_cropped_point_cloud:
  pclCombined = np.concatenate(pcl, axis=0)
  pclFrame = C.addFrame('pcl', 'world')
  pclFrame.setPointCloud(pclCombined, pclColor)

  C.view()

# 1.2.1 Antipodal Grasp Detection Algorithm

### Now we do the normals usoing open3d

In [127]:
# Reshape to a 2D array
pclCombined_reshaped = pclCombined.reshape(-1, 3)  # Collapse the first two dimensions into one

print(pclCombined_reshaped.shape)  # Should now be (1440 * 640, 3)

# Convert to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pclCombined_reshaped)

# #  Visualize the point cloud
# o3d.visualization.draw_geometries([pcd])
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=voxelSize)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# # print list of normals
# normals = np.asarray(downpcd.normals)
# print(normals)

(921600, 3)
Downsample the point cloud with a voxel of 0.05


### Now we do the clustering using kmeans from sklearn

In [128]:
# Cluster using kmeans from sklearn, there are 3 objects in the point cloud so we will use 3 clusters
kmeans = KMeans(n_clusters=numOfObjects, random_state=0).fit(np.asarray(downpcd.points))
print(kmeans.labels_)

# Visualize the clusters
clusters = []
for i in range(numOfObjects):
  cluster = downpcd.select_by_index(np.where(kmeans.labels_ == i)[0])
  cluster.paint_uniform_color([(i == 0), (i == 1), (i == 2)])
  
  points = np.asarray(cluster.points)
  print("Cluster", i, "has", points.shape[0], "points")
  
  # Use Local Outlier Factor to detect outliers
  lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
  is_inlier = lof.fit_predict(points) > 0  # Inliers are labeled as 1, outliers as -1
  
  # Filter out outliers
  filtered_points = points[is_inlier]
  cluster.points = o3d.utility.Vector3dVector(filtered_points)
  
  # generate normals for the filtered points
  cluster.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
  clusters.append(cluster)
  

  
# Check if normals are estimated
if downpcd.has_normals():
    print("Normals are already estimated.")
else:
    print("Normals are not estimated yet. Estimating now...")
    downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    print("Normals estimated.")

# Verify magnitudes of normals
normals = np.asarray(downpcd.normals)
magnitudes = np.linalg.norm(normals, axis=1)
print("Normal magnitudes (first 10):", magnitudes[:10])
print("Are normals normalized? ", np.allclose(magnitudes, 1.0, atol=1e-6))

[2 2 2 ... 1 3 3]
Cluster 0 has 1517 points
Cluster 1 has 368 points
Cluster 2 has 360 points
Cluster 3 has 765 points
Normals are already estimated.
Normal magnitudes (first 10): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Are normals normalized?  True


/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [129]:
# Fix normals orientation by aligning them towards a camera position
def reorient_normals(cluster):
    print("Reorienting normals...")
    viewpoint = np.array([0, 0, 10])  # Replace with the actual camera or world origin
    cluster.orient_normals_towards_camera_location(camera_location=viewpoint)

# Reorient normals for each cluster
for i, cluster in enumerate(clusters):
    print(f"Fixing normals for Cluster {i+1}...")
    reorient_normals(cluster)

if want_to_see_normals_of_objects:
    # Visualize the corrected normals
    for i, cluster in enumerate(clusters):
        print(f"Visualizing corrected normals for Cluster {i+1}...")
        o3d.visualization.draw_geometries([cluster], point_show_normal=True)


Fixing normals for Cluster 1...
Reorienting normals...
Fixing normals for Cluster 2...
Reorienting normals...
Fixing normals for Cluster 3...
Reorienting normals...
Fixing normals for Cluster 4...
Reorienting normals...


### Get the centre of point clouds of each cluster

In [130]:

centre = [None] * numOfObjects

for i, cluster in enumerate(clusters):
  bounding_box = cluster.get_axis_aligned_bounding_box()
  centre[i] = bounding_box.get_center()
  # print(centre[i])
  # print(f"Visualizing Cluster {i+1} with bounding box...")
  # o3d.visualization.draw_geometries([cluster, bounding_box])
  
  for point in clusters[i].points:
    point[0] = point[0] - centre[i][0]
    point[1] = point[1] - centre[i][1]
    point[2] = point[2] - centre[i][2]


# # First we will create frames for the objects
# sprayBottle = C.addFrame("pcl_sprayBottle")
# sprayBottle.setPointCloud(clusters[0].points, [255, 0, 0])
# sprayBottle.setPosition(centre[0])

# rollingPin = C.addFrame("pcl_rollingPin")
# rollingPin.setPointCloud(clusters[1].points, [0, 255, 0])
# rollingPin.setPosition(centre[1])

# soapBox = C.addFrame("pcl_soapBox")
# soapBox.setPointCloud(clusters[2].points, [255, 255, 0])
# soapBox.setPosition(centre[2])

# ballAdditional = C.addFrame("pcl_ballAdditional")
# ballAdditional.setPointCloud(clusters[3].points, [255, 255, 0])
# ballAdditional.setPosition(centre[3])

# C.addFrame("vis_soap", "pcl_soapBox").setShape(ry.ST.marker, [1]).setPosition(soapBox.getPosition())
C.view()

0

### Now we find antipodal grasp points

In [131]:
def find_antipodal_grasp_pairs(cluster):
    points = np.asarray(cluster.points)
    normals = np.asarray(cluster.normals)
    
    grasp_pairs = []
    
    # Iterate through all point pairs
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            # Compute the angle between normals
            normal_i = normals[i]
            normal_j = normals[j]
            dot_product = np.dot(normal_i, normal_j)
            angle = np.arccos(dot_product)  # Angle in radians
            
            # Check if angle is close to 180 degrees (antipodal condition)
            if np.abs(np.degrees(angle) - 180) < degToleration:  # 10-degree tolerance
                grasp_pairs.append((i, j))
    
    return grasp_pairs

def visualize_grasp_pairs(cluster, grasp_pairs):
    points = np.asarray(cluster.points)
    lines = []
    line_colors = []

    # Create lines connecting grasp pairs
    for pair in grasp_pairs:
        idx1, idx2 = pair
        lines.append([idx1, idx2])
        line_colors.append([1, 0, 0])  # Red lines for grasp pairs

    # Create Open3D line set for visualization
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(line_colors)

    # Visualize
    print("Visualizing grasp pairs...")
    o3d.visualization.draw_geometries([cluster, line_set])

def filter_opposite_side_grasps(cluster, grasp_pairs):
    points = np.asarray(cluster.points)
    normals = np.asarray(cluster.normals)

    # Compute the bounding box of the object
    min_bound = np.min(points, axis=0)
    max_bound = np.max(points, axis=0)
    object_extent = max_bound - min_bound

    # Find the principal axis (longest dimension) of the object
    # principal_axis_index = np.argmax(object_extent)  # 0 for x, 1 for y, 2 for z
    principal_axis_index = 0  # 0 for x, 1 for y, 2 for z

    valid_grasps = []

    for pair in grasp_pairs:
        idx1, idx2 = pair
        point1, point2 = points[idx1], points[idx2]
        normal1, normal2 = normals[idx1], normals[idx2]

        # Check if the vector between the grasp points aligns with the principal axis
        grasp_vector = point2 - point1
        grasp_vector_unit = grasp_vector / np.linalg.norm(grasp_vector)
        axis_vector = np.zeros(3)
        axis_vector[principal_axis_index] = 1

        alignment = np.abs(np.dot(grasp_vector_unit, axis_vector))  # Should be close to 1
        if alignment < 0.9:  # Adjust threshold if necessary
            continue

        # Check if the normals face opposite directions
        dot_product = np.dot(normal1, normal2)
        if dot_product > -0.9:  # Ensure normals are nearly opposite
            continue

        # Add this pair as valid if both conditions are satisfied
        valid_grasps.append(pair)

    return valid_grasps


In [132]:
# Process each cluster
grasp_pairs = [None] * len(clusters)
common_distance = [None] * len(clusters)
for i, cluster in enumerate(clusters):
    print(f"Processing Cluster {i+1}...")
    
    grasp_pairs[i] = find_antipodal_grasp_pairs(cluster)
    print(f"Number of potential grasp pairs in Cluster {i+1}: {len(grasp_pairs)}")
    
    grasp_pairs[i] = filter_opposite_side_grasps(cluster, grasp_pairs[i])
    print(f"Filtered grasp pairs for Cluster {i+1}: {len(grasp_pairs[i])}")

    
    # print(f"Visualizing grasp pairs for Cluster {i+1}...")
    # visualize_grasp_pairs(cluster, grasp_pairs[i])
    
# Finally, we need to select the best grasp pair for each object
# We will select the pair with the avg distance between the two points
# This is a simple heuristic, and there are more sophisticated methods available

best_pairs = [None] * len(clusters)
for i, cluster in enumerate(clusters):
    points = np.asarray(cluster.points)
    grasp_pair = grasp_pairs[i]
    distances = []
    
    # Calculate distances for all grasp pairs
    for pair in grasp_pair:
        idx1, idx2 = pair
        point1, point2 = points[idx1], points[idx2]
        distance = np.linalg.norm(point2 - point1)
        distances.append(distance)
    
    # Calculate the average distance
    avg_distance = np.mean(distances)
    best_pair = None
    min_diff = float('inf')
    
    # Find the pair with the distance closest to the average distance
    for pair, distance in zip(grasp_pair, distances):
        diff = abs(distance - avg_distance)
        if diff < min_diff:
            min_diff = diff
            best_pair = pair
    
    common_distance[i] = avg_distance
    best_pairs[i] = best_pair

    print(f"Best grasp pair for Cluster {i+1}: {best_pair} with distance closest to average {avg_distance}")
    
if want_to_see_best_grasp_for_each_object:
    # Visualize the best grasp pair
    best_pair = np.asarray(best_pair)
    best_pair = best_pair.reshape(-1, 2)
    visualize_grasp_pairs(cluster, best_pair)


Processing Cluster 1...


/tmp/ipykernel_55312/2628657640.py:14: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(dot_product)  # Angle in radians


Number of potential grasp pairs in Cluster 1: 4
Filtered grasp pairs for Cluster 1: 189
Processing Cluster 2...
Number of potential grasp pairs in Cluster 2: 4
Filtered grasp pairs for Cluster 2: 53
Processing Cluster 3...
Number of potential grasp pairs in Cluster 3: 4
Filtered grasp pairs for Cluster 3: 5
Processing Cluster 4...
Number of potential grasp pairs in Cluster 4: 4
Filtered grasp pairs for Cluster 4: 90
Best grasp pair for Cluster 1: (497, 1242) with distance closest to average 0.07315048466477161
Best grasp pair for Cluster 2: (125, 320) with distance closest to average 0.04188608402982608
Best grasp pair for Cluster 3: (60, 199) with distance closest to average 0.05902500893780619
Best grasp pair for Cluster 4: (436, 619) with distance closest to average 0.0480100365511025


# 1.2.2 Now we do the KOMO problem, ig we run all the points and try to get the feasible ones?

In [133]:
# Make a frame for each best pair
frames = [None] * len(clusters)
for i, cluster in enumerate(clusters):
  midpoint = (clusters[i].points[best_pairs[i][0]] + clusters[i].points[best_pairs[i][1]]) / 2
  
  # Re adjust the midpoint
  midpoint = midpoint + centre[i]
  # print("Grasp pair for cluster", i, ":", clusters[i].points[best_pairs[i][0]], "and", clusters[i].points[best_pairs[i][1]])
  # print("Midpoint:", midpoint)
  
  
  frame = C.addFrame("grasp_marker" + str(i))
  frame.setShape(ry.ST.marker, [.1])
  frame.setPosition(midpoint)
  # frame.setColor([1, 0, 0])
  
# We will also need to move the object to some point above the bin to avoid hitting the bin walls
bufferPointPos = C.getFrame("bin").getPosition()
# bufferPointPos[0] = bufferPointPos[0] - 0.5
bufferPointPos[1] = bufferPointPos[1] - 0.35
bufferPointPos[2] = bufferPointPos[2] + 0.6

buffer_point_frame = C.addFrame("buffer_point")
buffer_point_frame.setShape(ry.ST.marker, [.1])
buffer_point_frame.setPosition(bufferPointPos)


# also calc width of each object
width = [None] * len(clusters)
for i, cluster in enumerate(clusters):
    points = np.asarray(cluster.points)
    width[i] = np.linalg.norm(points[best_pairs[i][0]] - points[best_pairs[i][1]])

  
C.view()

0

In [134]:
try:
  del bot
  del komo
except:
  pass

In [ ]:
# Initialise BotOp and KOMO 
try:
  del bot
  del komo
except:
  pass
C.setJointState(qHome)

# Initialize BotOp for robot operation and Komo
bot = ry.BotOp(C, useRealRobot=False)
qHomeBot = bot.get_qHome()

# komo = ry.KOMO(
#   config=C, 
#   phases=1, 
#   slicesPerPhase=20, 
#   kOrder=0, 
#   enableCollisions=True
# )







for i in range(numOfObjects):
  komo = ry.KOMO(C, 1, 1, 0, False)
  
  # Define KOMO for picking up object
  komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
  komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'grasp_marker' + str(i)], ry.OT.eq, [1e1], [0,0, -0.02]) #constraint: gripper position

  # align gripper to grasp points
  komo.addObjective(
      times=[],
      feature=ry.FS.scalarProductXX,
      frames=['l_gripper', 'grasp_marker' + str(i)],
      type=ry.OT.eq,
      scale=[1e1],
      target=[1]
  )

  komo.addObjective(
      times=[],
      feature=ry.FS.scalarProductYY,
      frames=['l_gripper', 'grasp_marker' + str(i)],
      type=ry.OT.eq,
      scale=[1e1],
      target=[1]
  )

  komo.addObjective(
      times=[],
      feature=ry.FS.scalarProductZZ,
      frames=['l_gripper', 'grasp_marker' + str(i)],
      type=ry.OT.eq,
      scale=[1e1],
      target=[1]
  )

  ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
  print(ret)
  if not ret.feasible:
      raise Exception("No feasible path found for object " + str(i))

  # Retrieve the optimized path
  path = komo.getPath()[0]

  bot.moveTo(path, timeCost=1)

  while bot.getTimeToEnd()>0:
      bot.sync(C, .1)


  
  # grab the object using gripper
  bot.gripperMove(ry._left, width=width[i]-0.07, speed=1)


  # Intermediary step: Move the object to the buffer point

  try:
    del komo
  except:
    pass
  komo = ry.KOMO(C, 1, 1, 0, False)
  komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
  komo.addObjective(
    times=[], 
    feature=ry.FS.positionDiff, 
    frames=['l_gripper', 'buffer_point'], 
    type=ry.OT.eq, scale=[1e1]
  )

  ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
  intermediary = komo.getPath()[0]

  bot.moveTo(intermediary, timeCost=botOpSpeed)
  while bot.getTimeToEnd()>0:
      bot.sync(C, .1)





  # Now, we will move the object to the bin
  try:
    del komo
  except:
    pass

  komo = ry.KOMO(C, 1, 1, 0, False)
  komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome

  komo.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', 'bin-2'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[0, 0, 0.05]
  )

  ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
  path_to_bin2 = komo.getPath()[0]

  bot.moveTo(path_to_bin2, timeCost=botOpSpeed)

  while bot.getTimeToEnd()>0:
      bot.sync(C, .1)
    
  bot.gripperMove(ry._left, width=0.075, speed=1)
  while not bot.gripperDone(ry._left):
      bot.sync(C, .1)
      

  bot.moveTo(qHome, timeCost=5)
  while bot.getTimeToEnd()>0:
      bot.sync(C, .1)

  # # now we move back to buffer point
  # try:
  #   del komo
  # except:
  #   pass
  # komo = ry.KOMO(C, 1, 1, 0, False)
  # komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
  # komo.addObjective(
  #   times=[], 
  #   feature=ry.FS.positionDiff, 
  #   frames=['l_gripper', 'buffer_point'], 
  #   type=ry.OT.eq, scale=[1e1]
  # )

  # ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
  # intermediary = komo.getPath()[0]

  # bot.moveTo(intermediary, timeCost=botOpSpeed)
  # while bot.getTimeToEnd()>0:
  #     bot.sync(C, .1)
  
    
    
    

    
    
    
    
    
    

C.view()

{ time: 0.000902, evals: 18, done: 1, feasible: 1, sos: 0.0405865, f: 0, ineq: 0, eq: 0.00149871 }


### Now KOMO move to other bin

In [ ]:
# bot.moveTo(qHome, timeCost=botOpSpeed)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

In [ ]:
# speed = 0.1
# C.view()
# qHomeJS = C.getJointState()
# gripper_orig_pos = C.getFrame("l_gripper").getPosition()
# gripper_orig_quat = C.getFrame("l_gripper").getQuaternion()
# print(C.getFrameNames())

# # initiliase bot and save home position
# bot = ry.BotOp(C, useRealRobot=False)
# qHome = bot.get_qHome()